In [2]:
import altair as alt
import eco_style
import pandas as pd
alt.themes.enable('light')

ThemeRegistry.enable('light')

# Fig 1

In [47]:
# Load the data from the provided Excel file
data = pd.read_excel("fig1.xlsx")

# Prepare the data for plotting
data = data.rename(columns={'wb_countryname': 'Reference Country', 'r_squared_gdp_per_capita': 'Co-movement'})
data = data.sort_values(by='Co-movement')

colors = {
    'United States': 'rgb(239, 196, 93)',
    'Euro Zone': 'rgb(94, 179, 177)',
    'South America': 'rgb(211, 57, 80)'
}

data['color'] = data['Reference Country'].apply(lambda x: colors.get(x, 'rgb(168, 192, 222)'))

data['order'] = data['Reference Country'].apply(lambda x: 1 if x in ['United States', 'Euro Zone', 'South America'] else 0)

# Create the scatter plot with Altair
chart = alt.Chart(data).mark_circle().encode(
    x=alt.X('Reference Country:N', sort="y", title='Reference Country', axis=alt.Axis(labelExpr='')),
    y=alt.Y('Co-movement:Q', title='Co-movement'),
    color=alt.Color('color:N', scale=None, legend=None),
    tooltip=[
        {'title': 'Reference Country', 'field': 'Reference Country'},
        {'title': 'Co-movement', 'field': 'Co-movement', 'format': '.2f'}
    ],
    order='order',
    size=alt.condition(alt.datum['order'] == 1, alt.value(40), alt.value(20))
).properties(
    width=550,
    height=300
)

# Add specific annotations for the United States, Euro Zone, and South America
annotations = [
    {'Reference Country': 'United States', 'Co-movement': data[data['Reference Country'] == 'United States']['Co-movement'].values[0], 'color': 'blue'},
    {'Reference Country': 'Euro Zone', 'Co-movement': data[data['Reference Country'] == 'Euro Zone']['Co-movement'].values[0], 'color': 'red'},
    {'Reference Country': 'South America', 'Co-movement': data[data['Reference Country'] == 'South America']['Co-movement'].values[0], 'color': 'green'}
]

annotation_layer = alt.Chart(pd.DataFrame(annotations)).mark_text(align='left', 
                                                                  dy=-10,
                                                                  angle=350,
                                                                  baseline='middle', 
                                                                  dx=0).encode(
    x=alt.X('Reference Country:N', sort="y", title='Reference Country'),
    y='Co-movement:Q',
    text='Reference Country:N',
    color=alt.Color('Reference Country', legend=None),
)

# Combine the scatter plot and annotations
final_chart = chart + annotation_layer


final_chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [48]:
final_chart.save("fig1.json")
final_chart.save("fig1.png", scale_factor=2.0)  

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

# Fig2 

In [60]:
df = pd.read_csv("fig2.csv")
df = df.iloc[:, 1:]
df = df.melt(id_vars=["date"], var_name="series", value_name="value")
df = df.dropna()
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df['series'] = df.series.map({
    'arg_index' : "Price of Exports Index in Argentina", 
    'dollar_index' : 'Real Broad Dollar Index',
    'bra_index' : 'Real broad effective exchange rate in Brazil',
})
    
labels = {
    'Price of Exports Index in Argentina' : {
        'label_1' : 'Price of Exports Index',
        'label_2' : 'in Argentina'
    },
    'Real Broad Dollar Index' : {
        'label_1' : 'Real Broad Dollar Index',
        'label_2' : ''
    },
    'Real broad effective exchange rate in Brazil' : {
        'label_1' : 'Real broad effective',
        'label_2' : 'exchange rate in Brazil'
    }
}

series_max_dates = df.groupby('series')['date'].max().reset_index()

# add a label to 2023-08-01
df['label_1'] = df.apply(lambda x: labels[x['series']]['label_1'] if x['date'] == series_max_dates[series_max_dates['series'] == x['series']]['date'].values[0] else '', axis=1)
df['label_2'] = df.apply(lambda x: labels[x['series']]['label_2'] if x['date'] == series_max_dates[series_max_dates['series'] == x['series']]['date'].values[0] else '', axis=1)

In [63]:
base = alt.Chart(df).encode(
    x=alt.X("date:T"),
    y=alt.Y("value:Q", title=None),
    color=alt.Color(
        "series:N", 
        title=None,
        legend=None
    )
)

events = pd.DataFrame([
    {
        "start": "1998-01-01",
        "end" : "1999-03-02",
        "label_1" : "Asian and Russian",
        "label_2" : "Crises"
    },
    {
     "start" : "2008-06-13",
     "end" : "2009-02-17",
    "label_1" : "Global Financial",
    "label_2" : "Crisis"
    },
    {
        "start": "2014-07-02",
        "end" : "2015-09-01",
        "label_1" : "End of Commodity",
        "label_2" : "Boom"
    },
    {
        "start": "2020-01-01",
        "end" : "2021-01-01",
        "label_1" : "COVID-19",
        "label_2" : ""
    }
])

# add the shaded regions for the events
rects = alt.Chart(events).mark_rect(opacity=0.1).encode(
    x='start:T',
    x2='end:T',
    color=alt.value('gray'),
)

# add the text labels for the events
event_labels_1 = alt.Chart(events).mark_text(dy=-17, dx=0, align='left').encode(
    x='start:T',
    y=alt.value(0),
    text='label_1:N'
)

event_labels_2 = alt.Chart(events).mark_text(dy=-7, dx=0, align='left').encode(
    x='start:T',
    y=alt.value(0),
    text='label_2:N',
)



lines = base.mark_line()

line_labels_1 = base.mark_text(dy=0, dx=5, align='left').encode(
    text='label_1:N'
)

line_labels_2 = base.mark_text(dy=10, dx=5, align='left').encode(
    text='label_2:N'
)

chart =  rects + lines + event_labels_1 + event_labels_2 + line_labels_1 + line_labels_2

chart = chart.properties(
    width=600,
    height=400
)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [64]:
chart.save("fig2.json")
chart.save("fig2.png", scale_factor=3)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

In [59]:
df.drop_duplicates(subset=['series'], keep='last')

,date,series,value,label_1,label_2
361,2024-02-01,Real Broad Dollar Index,115.111400,,
723,2024-02-01,Real broad effective exchange rate in Brazil,94.740634,,
1079,2023-08-01,Price of Exports Index in Argentina,172.222634,Price of Exports Index,in Argentina


In [32]:
events

,start,end,label
0,1998-01-01,1992-02-28,Asian and Russian Crises
1,2008-06-13,2009-02-17,Global Financial Crisis
2,2014-07-02,2015-09-01,End of Commodity Boom
3,2020-01-01,2021-01-01,COVID-19


In [33]:
start_date = pd.to_datetime("1998-01-01")
# add 1.16 years to the start date
end_date = start_date + pd.DateOffset(days=365*(70/180)*3)
end_date

Timestamp('1999-03-02 00:00:00')

In [12]:
df.series.unique()

array(['dollar_index', 'bra_index', 'arg_index'], dtype=object)